In [39]:
source ../modules/setup.tcl
set year 2020
set day 18
aoc::get-puzzle $year $day 1
aoc::get-puzzle $year $day 2
set input [string trim [aoc::get-input $year $day]]
jupyter::html "<h2>Input</h2>"
jupyter::display "text/plain" [string range $input 0 100]...;

(cached)

--- Day 18: Operation Order --- As you look out the window and notice a heavily-forested continent slowly appear over the horizon, you are interrupted by the child sitting next to you. They're curious if you could help them with their math homework. Unfortunately, it seems like this "math" follows different rules than you remember. The homework (your puzzle input) consists of a series of expressions that consist of addition ( + ), multiplication ( * ), and parentheses ( (...) ). Just like normal math, parentheses indicate that the expression inside must be evaluated before it can be used by the surrounding expression. Addition still finds the sum of the numbers on both sides of the operator, and multiplication still finds the product. However, the rules of operator precedence have changed. Rather than evaluating multiplication before addition, the operators have the same precedence , and are evaluated left-to-right regardless of the order in which they appear. For example, the steps to evaluate the expression 1 + 2 * 3 + 4 * 5 + 6 are as follows: 
 1 + 2 * 3 + 4 * 5 + 6
 3 * 3 + 4 * 5 + 6
 9 + 4 * 5 + 6
 13 * 5 + 6
 65 + 6 71 
 Parentheses can override this order; for example, here is what happens if parentheses are added to form 1 + (2 * 3) + (4 * (5 + 6)) : 1 + (2 * 3) + (4 * (5 + 6))
 1 + 6 + (4 * (5 + 6))
 7 + (4 * (5 + 6) )
 7 + (4 * 11 ) 7 + 44 51 
 Here are a few more examples: 
 
 2 * 3 + (4 * 5) becomes 26 . 
 5 + (8 * 3 + 9 + 3 * 4 * 3) becomes 437 . 
 5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) becomes 12240 . 
 ((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 becomes 13632 . 
 Before you can help with the homework, you need to understand it yourself. Evaluate the expression on each line of the homework; what is the sum of the resulting values?

--- Part Two --- You manage to answer the child's questions and they finish part 1 of their homework, but get stuck when they reach the next section: advanced math. Now, addition and multiplication have different precedence levels, but they're not the ones you're familiar with. Instead, addition is evaluated before multiplication. For example, the steps to evaluate the expression 1 + 2 * 3 + 4 * 5 + 6 are now as follows: 
 1 + 2 * 3 + 4 * 5 + 6
 3 * 3 + 4 * 5 + 6
 3 * 7 * 5 + 6 3 * 7 * 11
 21 * 11 231 
 Here are the other examples from above: 
 
 1 + (2 * 3) + (4 * (5 + 6)) still becomes 51 . 
 2 * 3 + (4 * 5) becomes 46 . 
 5 + (8 * 3 + 9 + 3 * 4 * 3) becomes 1445 . 
 5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) becomes 669060 . 
 ((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 becomes 23340 . 
 What do you get if you add up the results of evaluating the homework problems using these new rules?

(cached)

Input

4 * ((9 + 9 * 5 + 5) + 3)
4 + 5 * (3 + 4 * 3 * 8 * 4 + (5 * 4 + 8)) * 6
3 * (6 + (6 * 9 * 9) + (3 + 7...

In [32]:
namespace import tcl::mathop::*
proc reduce {terms order} {
    set n [llength $terms]
    foreach ops $order {
        set n [llength $terms]
        set nterms {}
        for {set i 0} {$i<$n} {incr i} {
            set tok [lindex $terms $i] 
            if {$tok in $ops} {
                set first [lindex $nterms end]
                incr i
                set second [lindex $terms $i]
                set nterms [lreplace $nterms end end [$tok $first $second]]
            } else {
                lappend nterms [lindex $terms $i]
            }
        }
        set terms $nterms
    }
    return $terms
    
    
}

proc _calc {input order idx} {
    set phrase {}
    while {$idx < [string length $input]} {
        set c [string index $input $idx]
        incr idx
        if {$c eq " "} continue
        if {$c eq "("} {
            lassign [_calc $input $order $idx] part idx
            lappend phrase $part
            continue
        }
        if {$c eq ")"} {
            return [list [reduce $phrase $order] $idx]
        }
        lappend phrase $c
        
    }
    return [list $phrase $idx]
}
proc calc {input order} {
    lassign [_calc $input $order 0] phrase
    reduce $phrase $order
}

In [40]:
proc parts input {
   
    set result1  [+ {*}[lmap l [split $input \n] {calc $l {{+ *}}}]]
    set result2  [+ {*}[lmap l [split $input \n] {calc $l {+ *}}]]
    return [list $result1 $result2]
}
aoc::results 

Day1	11297104473091
Day2	185348874183674
78125 microseconds per iteration
